In [5]:
#kode kedua
import numpy as np
import os
from google.colab import files # Modul untuk upload dan download file

def process_multi_xyz(file_path):
    """
    Fungsi untuk membaca koordinat 3D dan mengubahnya menjadi
    Matriks Coulomb (Deskriptor ML).
    """
    all_features = []

    with open(file_path, 'r') as f:
        lines = f.readlines()

    i = 0
    total_lines = len(lines)
    conformer_count = 0

    while i < total_lines:
        try:
            # Mengambil jumlah atom
            num_atoms = int(lines[i].strip())

            # Mengambil baris atom (X, Y, Z)
            atom_data = lines[i+2 : i+2+num_atoms]

            coords = []
            atomic_nums = []

            # Mapping simbol atom ke nomor atom (Z)
            z_map = {'H': 1, 'C': 6, 'O': 8}

            for line in atom_data:
                parts = line.split()
                atomic_nums.append(z_map[parts[0]])
                coords.append([float(parts[1]), float(parts[2]), float(parts[3])])

            coords = np.array(coords)
            atomic_nums = np.array(atomic_nums)

            # Menghitung Matriks Coulomb (CM)
            cm = np.zeros((num_atoms, num_atoms))
            for r in range(num_atoms):
                for c in range(num_atoms):
                    if r == c:
                        # Diagonal: 0.5 * Z^2.4
                        cm[r, c] = 0.5 * (atomic_nums[r]**2.4)
                    else:
                        # Off-diagonal: Zi*Zj / Jarak
                        dist = np.linalg.norm(coords[r] - coords[c])
                        cm[r, c] = (atomic_nums[r] * atomic_nums[c]) / dist

            # Ubah matriks (47x47) menjadi satu baris (2209 kolom)
            all_features.append(cm.flatten())

            i += (num_atoms + 2)
            conformer_count += 1

        except (ValueError, IndexError):
            break

    return np.array(all_features), conformer_count

# --- PROSES UTAMA ---

print("Silahkan upload file .xyz Anda (contoh: enol_gas.xyz)")
uploaded = files.upload()

for nama_file in uploaded.keys():
    print(f"\nMemproses: {nama_file}...")

    # 1. Jalankan ekstraksi fitur
    X, jumlah = process_multi_xyz(nama_file)

    print("-" * 40)
    print(f"Selesai mengekstrak {jumlah} konformer.")

    # 2. Tentukan nama file output (.npy)
    nama_output = nama_file.replace(".xyz", ".npy")

    # 3. Simpan ke sistem file sementara Colab
    np.save(nama_output, X)
    print(f"Data disimpan sebagai: {nama_output}")

    # 4. DOWNLOAD OTOMATIS ke komputer kamu
    print(f"Mengunduh {nama_output}...")
    files.download(nama_output)
    print("-" * 40)

Silahkan upload file .xyz Anda (contoh: enol_gas.xyz)


Saving keto_sol.xyz to keto_sol.xyz

Memproses: keto_sol.xyz...
----------------------------------------
Selesai mengekstrak 990 konformer.
Data disimpan sebagai: keto_sol.npy
Mengunduh keto_sol.npy...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

----------------------------------------
